## 获取基金历史数据

Alex

Created on: 08/07/2021

In [4]:
import warnings

warnings.filterwarnings('ignore')

import datetime
import threading

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

today=datetime.datetime.now().date()    
start_day='2020-01-01'

funds_code=pd.read_csv('Data/基金代码名称对照表.csv')
funds_code=funds_code[~funds_code['基金名称'].str.contains('债')&~funds_code['基金名称'].str.contains('货币')&~funds_code['基金名称'].str.contains('一年')].基金代码.values
funds_code

array([     1,      6,      8, ..., 952009, 952035, 952099])

In [5]:
def multi_threads(func,file_list,threads):
    if __name__ == "__main__": 
        steps=int(len(file_list)/(threads))+1
        num=int(len(file_list)/steps)+1

        for k in range(num):
            locals()['t'+str(k)]=threading.Thread(target=func, 
                                                args=([file_list[steps*k:min(steps*(k+1),len(file_list))]])) 
        for k in range(num):
            locals()['t'+str(k)].start() 
        for k in range(num):
            locals()['t'+str(k)].join() 
            
            
def get_history(funds_code):
    for i,code in enumerate(funds_code):
        for i in range(21):
            url='http://quotes.money.163.com/fund/jzzs_%s_%s.html?start=%s&end=%s&sort=TDATE&order=desc'%(str(code).zfill(6),i,start_day,str(today))
            try:
                res = requests.get(url,timeout=3)
            except:break
            bs4=BeautifulSoup(res.text)
            try:bs4=bs4.find('tbody').find_all('tr')
            except:
                print('%s Failed'%code)
                break
            if len(bs4)==0:break
            try:
                # bs4=[x for x in bs4]
                dates=[x.find_all('td')[0].get_text() for x in bs4]
                net=[x.find_all('td')[1].get_text() for x in bs4]
                per_change=[x.find('span').get_text() for x in bs4]
                cc=[code for x in range(len(dates))]
            except:continue
            funds_res.append((np.vstack((cc,dates,net,per_change)).T))


In [6]:
%%time
# 从网易财经获取基金历史数据
funds_res=[]
# multi_threads(get_history,funds_code,100) # 爬取全部基金历史数据，耗时较长
multi_threads(get_history,funds_code[:100],100) # sample code，只爬取了前100个基金

CPU times: user 46.3 s, sys: 5.23 s, total: 51.5 s
Wall time: 56 s


In [7]:
# clean data and output
funds_res1=[x for x in funds_res if len(x)!=0]
funds_res1=np.array(funds_res1)
result=np.array((None,None,None,None,))
for x in funds_res1:
    result=np.vstack((result,x))

funds_res1=np.array(funds_res1)
funds_res1=[x[0] for x in funds_res1]
funds_history_new=pd.DataFrame(result,columns=['基金代码','日期','净值','涨跌幅'])
funds_history_new=funds_history_new.iloc[1:]
funds_history_new['基金代码']=funds_history_new.基金代码.astype(int).astype(str).transform(lambda x:x.zfill(6))
funds_history_new['涨跌幅']=funds_history_new.涨跌幅.transform(lambda x:x[:-1]).astype(float)
funds_history_new['涨跌幅']/=100
funds_history_new['涨跌幅']+=1

display(funds_history_new)
funds_history_new.to_csv('Data/funds_data.csv',index=False)


,基金代码,日期,净值,涨跌幅
1,000039,2021-08-06,4.3133,1.0091
2,000039,2021-08-05,4.2742,1.0086
3,000039,2021-08-04,4.2378,1.0604
4,000039,2021-08-03,3.9966,0.9550
5,000039,2021-08-02,4.1851,1.0153
...,...,...,...,...
37756,000342,2020-01-08,1.3670,0.9964
37757,000342,2020-01-07,1.3720,1.0007
37758,000342,2020-01-06,1.3710,1.0015
37759,000342,2020-01-03,1.3690,1.0029
